# Data Science Notebook 

In [1]:
import os, sys
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn import set_config
from loguru import logger

In [2]:
# Turn off jedi
%config IPCompleter.use_jedi=False

# Set sklearn output to pandas
set_config(transform_output = "pandas")

workdir = os.path.dirname(os.getcwd())

sys.path.append(workdir)

pd.options.display.max_columns = None

warnings.filterwarnings("ignore")

# display(
#     HTML(
#         """
#         <style>
#         .container { width:100% !important; }
#         </style>
#         """
#     )
# )

In [3]:
import os
from basix import files, parquet
from src.data.load import get_links_by_types, download_file, extract_filename, extract_type, read_csv_within_zip
from src.config import PATH_ZIP_DIRECTORY, PATH_PARQUET_DIRECTORY
from src.data.load import add_date, get_current_timestamp

update = True

timestamp = get_current_timestamp()

urls = get_links_by_types()

database_type = 'Empresas'

for database_type in urls:
    logger.info(f"################## {database_type}")
    
    for url in urls[database_type][0:2]:
        try:

            filename = extract_filename(url)

            logger.info(f"---------------------- {filename.replace(".zip", "")}")

            zip_directory_path = os.path.join(PATH_ZIP_DIRECTORY, database_type)

            zip_local_filepath = os.path.join(zip_directory_path, filename)

            parquet_directory_path = os.path.join(PATH_PARQUET_DIRECTORY, database_type)

            parquet_local_filepath = os.path.join(parquet_directory_path, filename.replace(".zip", ".parquet"))

            files.make_directory(zip_directory_path)

            if update:
                logger.info(f"Downloading zip from {url} and persisting to {zip_local_filepath}")
                download_file(url, zip_local_filepath)

            logger.info(f"Reading csv within {zip_local_filepath}")
            df = read_csv_within_zip(zip_local_filepath)

            df = df.pipe(add_date, timestamp = timestamp)

            logger.info(f"Persisting dataframe to parquet format in {parquet_local_filepath}")
            parquet.write(df, parquet_local_filepath, overwrite=True)

        except Exception as err:
            logger.error(err)

2023-02-10 11:25:15.250 | INFO     | __main__:<cell line: 15>:29 - Downloading zip from http://200.152.38.155/CNPJ/Empresas0.zip and persisting to datasets/raw/zip/Empresas/Empresas0.zip
2023-02-10 11:29:19.906 | INFO     | __main__:<cell line: 15>:32 - Reading csv within datasets/raw/zip/Empresas/Empresas0.zip
2023-02-10 11:29:36.512 | INFO     | __main__:<cell line: 15>:37 - Persisting dataframe to parquet format in datasets/raw/parquet/Empresas/Empresas0.parquet
2023-02-10 11:29:44.926 | INFO     | __main__:<cell line: 15>:29 - Downloading zip from http://200.152.38.155/CNPJ/Empresas1.zip and persisting to datasets/raw/zip/Empresas/Empresas1.zip
2023-02-10 11:31:09.670 | INFO     | __main__:<cell line: 15>:32 - Reading csv within datasets/raw/zip/Empresas/Empresas1.zip
2023-02-10 11:31:16.053 | INFO     | __main__:<cell line: 15>:37 - Persisting dataframe to parquet format in datasets/raw/parquet/Empresas/Empresas1.parquet


In [5]:
df_2 = pd.read_parquet('datasets/raw/parquet/Empresas/')

In [6]:
df_2

16385211

In [7]:

parquet_directory_path = os.path.join(PATH_PARQUET_DIRECTORY, database_type)

parquet_local_filepath = os.path.join(zip_directory_path, filename)

parquet_local_filepath

'data/raw/zip/Empresas/Empresas1.zip'

In [34]:
from datetime import datetime
def add_date(df: pd.DataFrame) -> pd.DataFrame:
    
    df["creation_date"] = datetime.strftime(datetime.today(), "%Y%m%d")
    
    return df

In [ ]:
files.make_directory(zip_directory_path)

In [4]:
df.sample(3)

,cnpj_base,razao_social,natureza_juridica,qualificacao,capital_social,porte_empresa,ente_federativo
2802612,2891503,ASSOCIACAO MORADORES AGROVILA MONTE ALTO,3999,16,"0,00",5.0,NaN
2813093,2902033,M RIBAS CONSULTORES ASSOCIADOS LTDA,2062,49,"0,00",1.0,NaN
3822229,3942771,SHIRATORI E RODRIGUES BAZAR LTDA,2062,49,"0,00",1.0,NaN


In [5]:
import pandas as pd
import zipfile

zipfile_path = 

zf = zipfile.ZipFile('data/raw/Empresas0.zip') 
df = pd.read_csv(zf.open('intfile.csv'))

In [5]:
from src.config import DATABASE_COLUMNS

DATABASE_COLUMNS[database_type]

['cnpj_base',
 'razao_social',
 'natureza_juridica',
 'qualificacao',
 'capital_social',
 'porte_empresa',
 'ente_federativo']

In [6]:
%pwd

'/home/eduardo/Documentos/Projetos/CNPJ'

In [12]:
import zipfile
from src.config import DATABASE_COLUMNS

def read_csv_within_zip(zip_filepath: str) -> pd.DataFrame:
    """
    This function reads all csv files within a zip file and concatenates them into a single dataframe.
    
    Args:
        zip_filepath (str): the file path to the zip file.

    Returns:
        pd.DataFrame: a dataframe containing the concatenated data from all csv files within the zip file.
    """

    zf = zipfile.ZipFile(zip_filepath)

    database_type = os.path.dirname(local_filepath).split("/")[-1]

    pdread_opts = dict(sep=";", encoding="latin-1", header=None)

    dataframe = pd.DataFrame()

    for filename in zf.namelist():
        try:
            temp = pd.read_csv(zf.open(filename), **pdread_opts)
            temp.columns = DATABASE_COLUMNS[database_type]
            dataframe = pd.concat(
                [
                    dataframe,
                    temp
                ]
            )

        except Exception as err:
            logger.warning(
                f"It was not possible to read the file {filename}"
                f"within de zip file {local_filepath}"
            )
            
    return dataframe

In [15]:
dataframe['porte_empresa']

0           1.0
1           1.0
2           1.0
3           1.0
4           1.0
           ... 
11890346    1.0
11890347    1.0
11890348    5.0
11890349    1.0
11890350    5.0
Name: porte_empresa, Length: 11890351, dtype: float64

In [12]:
[
    "cnpj_base", 
    "razao_social", 
    "natureza_juridica", 
    "qualificacao", 
    "capital_social", 
    "porte_empresa", 
    "ente_federativo"
]

,0,1,2,3,4,5,6
0,41273597,PACHARRUS QUEIROZ DA COSTA E SILVA 03618384335,2135,50,"5000,00",1.0,NaN
1,41273598,GLORIA VIANA DIAS DA SILVA 13118961716,2135,50,"1100,00",1.0,NaN
2,41273599,ANA PAULA DA SILVA DE PAULA 04659802957,2135,50,"2000,00",1.0,NaN
3,41273600,AVANILSON BRUNO MATIAS DA SILVA 08778601495,2135,50,"50000,00",1.0,NaN
4,41273601,GABRIELA HELENA FACINI DA SILVA 47022415838,2135,50,"2000,00",1.0,NaN
...,...,...,...,...,...,...,...
11890346,98819550,ANTONIO ORIQUES CARDOSO,2135,50,"0,00",1.0,NaN
11890347,98819568,ELI ORIQUES CARDOSO,2135,50,"0,00",1.0,NaN
11890348,98819600,PAULINO LEMOS DA SILVA,2135,50,"0,00",5.0,NaN
11890349,98819832,JOAO FELTRIN,2135,50,"0,00",1.0,NaN


In [21]:


from bs4 import BeautifulSoup, SoupStrainer
import requests


def get_links():
    page = requests.get(BASE_URL)    
    data = page.text
    soup = BeautifulSoup(data)

    for link in soup.find_all('a'):
        if str(link.get('href')).endswith('.zip'): 
            cam = link.get('href')
            # if cam.startswith('http://http'):
            #     cam = 'http://' + cam[len('http://http//'):] 
            if not cam.startswith('http'):
                yield BASE_URL+cam
            else:
                yield cam

In [24]:
links = list(get_links())

In [13]:
from src.config import DATABASE_TYPES

{k: None for k in DATABASE_TYPES}

{'Cnaes': None,
 'Empresas': None,
 'Estabelecimentos': None,
 'Motivos': None,
 'Municipios': None,
 'Naturezas': None,
 'Paises': None,
 'Qualificacoes': None,
 'Simples': None,
 'Socios': None}